# Imports

In [2]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

In [3]:
import time
import logging
import pandas as pd
from datetime import datetime, timedelta, timezone
from dateutil.relativedelta import relativedelta
import plaid


# gcp
from google.cloud import bigquery
from google.cloud import pubsub_v1
from google.cloud import scheduler_v1
from google.cloud import functions_v2
from google.cloud.functions_v1.types import ListFunctionsRequest
from google.protobuf import duration_pb2

# SCHEMAS
from jobs.bq_table_schemas import BqTableSchemas
from jobs.cloud_schemas import CloudSchemas


# UTILS
from utils.bq_utils import BqUtils
from utils.plaid_utils import PlaidUtils
from utils.secrets_utils import SecretsUtils
from utils.crypto_utils import CryptoUtils
from utils.budget_values import BudgetValues
from utils.financial_accounts import FinancialAccounts
from utils.plaid_transactions import PlaidTransactions
from utils.plaid_investments import PlaidInvestments


# get all secrets
sec = SecretsUtils()
secrets = sec.create_secrets_dict(plaid_secret_type="PROD")
PLAID_CLIENT_ID = secrets["PLAID_CLIENT_ID"]
PLAID_SECRET = secrets["PLAID_SECRET_PROD"]
PLAID_ACCESS_TOKENS = sec.get_access_token_secrets(secrets)
PLAID_HOST = plaid.Environment.Production
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
CRYPTO_SECRETS = secrets["CRYPTO_SECRETS"]

# initialize clients
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
financial_accounts = FinancialAccounts(bq_client, plaid_client)
plaid_transactions = PlaidTransactions(bq_client, plaid_client)
plaid_investments = PlaidInvestments(bq_client, plaid_client)
budget_values = BudgetValues(bq_client)
crypto = CryptoUtils()
bq_tables = BqTableSchemas()

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 1000)

# Budget Values

In [4]:
for i in range(-18, 1):
    # print(bq.get_partition_date(i, partition_format="YYYYMM"))
    budget_values.upload_budget_values_df_to_bq(offset=i)

SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202212`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202301`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202302`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202303`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202304`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202305`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202306`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202307`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202308`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202309`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202310`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202311`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202312`
SUCCESS: df uploaded to `zsc-personal.